In [ ]:
!apt-get install yasm

Install ffmpeg and libavcodec-dev (plus related stuff) from
https://launchpad.net/~mc3man/+archive/ubuntu/trusty-media

Install OpenCV using the instructions from 
http://www.pyimagesearch.com/2015/06/22/install-opencv-3-0-and-python-2-7-on-ubuntu/

In my case I had a build error related to missing CUDA libs, so the build command I used was

    cmake -D CMAKE_BUILD_TYPE=RELEASE \
            -D CMAKE_INSTALL_PREFIX=/usr/local \
            -D INSTALL_C_EXAMPLES=OFF \
            -D INSTALL_PYTHON_EXAMPLES=ON \
            -D "OPENCV_EXTRA_MODULES_PATH=../../opencv_contrib/modules" \
            -D BUILD_EXAMPLES=ON \
            -D WITH_CUDA=OFF ..



In [ ]:
!pip install pytube

In [ ]:
!mkdir videos

In [ ]:
from pytube import YouTube

# not necessary, just for demo purposes.
from pprint import pprint

yt = YouTube("https://www.youtube.com/watch?v=yim23IFJOyQ")

yt.set_filename('Chimpanzees1')
# Once set, you can see all the codec and quality options YouTube has made
# available for the perticular video by printing videos.

# pprint(yt.get_videos())

In [ ]:
video = yt.get('mp4', '360p')

In [ ]:
# In this case, we'll need to specify both the codec (mp4) and resolution
# (either 360p or 720p).

# Okay, let's download it!
video = yt.get('mp4', '360p')
video.download('./videos/')


In [ ]:
# Now let's try to read this using our fresh opencv build..

In [1]:
import cv2
import h5py


In [2]:
from IPython.display import HTML, FileLink, display
from base64 import b64encode
def link_video(path):
    src = FileLink(path)
    video_tag = '<video controls alt="test" src="%s">' % (path)
    
    display(HTML(data=video_tag))


In [3]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    # img should be an numpy ndarray of shape (width, heigth, channels)
    if (not ret):
        break
    if (shape is None):
        shape = img.shape
    if (dtype is None):
        dtype = img.dtype
    frames+=1
    
print "Could read %d frames of shape %r and dtype %r" % (frames, shape, dtype)

Could read 1490 frames of shape (360, 640, 3) and dtype dtype('uint8')


In [4]:
import h5py

h5f = h5py.File('videos/Chimpanzees1.hdf5', 'w')
h5g = h5f.create_group('input')

In [5]:
import numpy as np

In [6]:
ishape = tuple([ frames ] + list(shape))
ichunkshape = tuple([ 1 ] + list(shape))
imaxshape = tuple([ None ] + list(shape))

In [7]:
from numpy.random import permutation

In [8]:
inframes = h5g.create_dataset('raw_hsl_frames', 
                             shape=ishape, dtype=np.float32, 
                             chunks=ichunkshape, maxshape=imaxshape,
                             fletcher32=True, fillvalue=0)


In [9]:
permuted_indices = permutation(frames)

In [10]:
vc = cv2.VideoCapture('videos/Chimpanzees1.mp4')
frames = 0
shape = None
dtype = None
while (True):
    ret, img = vc.read()
    if (not ret):
        break
    hsvimg = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    #print hsvimg.dtype
    
    hsvimgf = hsvimg.astype(np.float32) / 255.0
    # img should be an numpy ndarray of shape (width, heigth, channels)
    inframes[permuted_indices[frames],:,:,:] = hsvimgf
    
    hsvimg2 = (inframes[permuted_indices[frames],:,:,:] * 255.0).astype(np.uint8)
    
    #print np.sum(hsvimg-hsvimg2)
    #bgrimg = cv2.cvtColor(hsvimg2, cv2.COLOR_HSV2BGR)
    #print np.sum(img-bgrimg)
    
    #cv2.imwrite("videos/recoded1_%05d.png" % (frames), bgrimg)
    
    frames+=1
    if (frames % 50==0):
        print "Frame #%d" % (frames)
print "Read %d frames" % (frames)


Frame #50
Frame #100
Frame #150
Frame #200
Frame #250
Frame #300
Frame #350
Frame #400
Frame #450
Frame #500
Frame #550
Frame #600
Frame #650
Frame #700
Frame #750
Frame #800
Frame #850
Frame #900
Frame #950
Frame #1000
Frame #1050
Frame #1100
Frame #1150
Frame #1200
Frame #1250
Frame #1300
Frame #1350
Frame #1400
Frame #1450
Read 1490 frames


In [ ]:
ls -hs videos/Chimpanzees1.hdf5

In [11]:
import numpy as np

In [ ]:
rm -rf videos/Chimpanzee1RegionTest_*

In [ ]:
!rm videos/Chimpanzees1_*.png
!ffmpeg -i videos/Chimpanzees1.mp4 -r 6 -f image2 videos/Chimpanzees1_%05d.png

In [ ]:
!ls videos

In [ ]:
!rm videos/Chimpanzees1_recoded.mp4
!ffmpeg -i videos/Chimpanzees1_%05d.png -b 512 -c:v libx264 -b:v 400k -r 10 -preset medium videos/Chimpanzees1_recoded.mp4

In [ ]:
!ls -lahs videos/Chimpanzees1*.mp4

In [ ]:
link_video('videos/Chimpanzees1_recoded.mp4')

In [12]:
import os
import h5py
from glob import glob



In [ ]:
!cat ~/.keras/keras.json


Note:

Read http://arxiv.org/pdf/1402.3337.pdf and use the ThresholdedReLu activation from http://keras.io/layers/advanced_activations/


In [63]:

import numpy as np
np.random.seed(1337)  # for reproducibility


from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.advanced_activations import ThresholdedReLU
from keras.layers.embeddings import Embedding
from keras.layers.convolutional import Convolution2D, ZeroPadding2D
from keras.datasets import imdb
from keras.optimizers import SGD
from keras.regularizers import l1,l2,l1l2
from keras import backend as K

In [14]:
import keras.layers.convolutional as kconv

In [98]:
print 'Build model...'

num_feature_detectors=16*16*3
tile_size=16
receptive_field_size=24
model = Sequential()
#model.add(Dropout(0.001, input_shape=ichunkshape[1:]))
w1 = np.zeros(shape=(receptive_field_size, receptive_field_size, 3, num_feature_detectors), dtype=np.float32)
b1 = np.zeros(shape=(num_feature_detectors,), dtype=np.float32)
off = (receptive_field_size - tile_size) / 2
for i in range(tile_size):
    for j in range(tile_size):
        for h in range(3):
            w1[off+i,off+j,h, i*3*tile_size+j*3+h] = 1.0

input_features = Convolution2D( name='encode_tiled_features', 
                        nb_filter=num_feature_detectors, nb_row=receptive_field_size,nb_col=receptive_field_size, 
                        subsample=(tile_size,tile_size), weights=[w1,b1],
                        border_mode='valid', trainable=False,
                        dim_ordering='tf', input_shape=ichunkshape[1:] )


model.add(input_features)
print K.get_value(input_features.params[0]).shape
print K.get_value(input_features.params[1]).shape
print w1.shape
#model.add(Activation('relu'))
decoder = Convolution2D( name='decode_tiled_features', 
                        nb_filter=tile_size*tile_size*3, nb_row=1,nb_col=1, subsample=(1,1),
                        border_mode='valid',
                        dim_ordering='tf')
model.add(decoder)
model.add(Reshape(dims=(decoder.output_shape[1]*tile_size,decoder.output_shape[2]*tile_size, 3)))
model.add(ZeroPadding2D(((ichunkshape[1]-model.output_shape[1])/2, (ichunkshape[2]-model.output_shape[2])/2), 
                        dim_ordering='tf'))

def mean_squared_error_all(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true), axis=None)



print model.output_shape

Build model...
(24, 24, 3, 768)
(768,)
(24, 24, 3, 768)
(None, 360, 640, 3)


In [99]:
opt = SGD(lr=0.01, momentum=0.6, decay=0.0, nesterov=True)

In [100]:
model.compile(optimizer=opt, loss=mean_squared_error_all, class_mode='binary')

In [57]:
print "OK"

OK


In [ ]:
def data_generator(batchsize):
    for i in range(10):
        for j in permutation(1300):
            bi = inframes[j:(j+batchsize),:,:,:]
            yield bi,  bi
    
for i in range(10):
    print "Over-Epoch %i" % (i)
    model.fit_generator(data_generator(3), validation_data=(inframes[1400:1500], inframes[1400:1500]),
              show_accuracy=False, nb_epoch=30, samples_per_epoch=200, nb_worker=1)

Over-Epoch 0
Epoch 1/30
198/200 [============================>.] - ETA: 0s - loss: 0.2636

In [37]:
for bi, ba in data_generator(2):
    print bi.shape
    print ba.shape

Batch at 390


TypeError: Argument sequence too long

In [ ]:
predictions = model.predict(inframes[1:4], batch_size=4)

In [ ]:
reverse_permute = np.array(permuted_indices, dtype=np.int32)
for i in range(frames):
    reverse_permute[permuted_indices[i]]=i


In [ ]:
j = 0
for i in permuted_indices:
    
    
    #print np.sum(hsvimg-hsvimg2)
    fframe = inframes[i,:,:,:]
    frame = (fframe * 255.0).astype(np.uint8)
    
    bgrimg = cv2.cvtColor(frame, cv2.COLOR_HSV2BGR)
    
    #print np.sum(img-bgrimg)
    cv2.imwrite("videos/recoded1_%05d.png" % (frames), bgrimg)
    
    pframe = model.predict(fframe[np.newaxis,:,:,:], batch_size=1)
    #print mean_squared_error_all
    piframe = (pframe[0,:,:,:]*255.0).astype(np.uint8)
    rgbimg = cv2.cvtColor(piframe, cv2.COLOR_HSV2BGR)
        
    j+=1
    cv2.imwrite("videos/reconstruction1_%05d.png" % (j), rgbimg)

    if (j % 10==0):
        me = np.mean(np.square((bgrimg.astype(np.float32)/255.0)-(rgbimg.astype(np.float32)/255.0)))
        print "Wrote image %d - MSE: %f" % (j, me)

In [ ]:
!ffmpeg -i videos/reconstruction1_%05d.png -b 512 -c:v libx264 -b:v 400k -r 10 -preset medium videos/Chimpanzees1_reconstruction.mp4

In [ ]:
!ls -hs videos/*.mp4

In [ ]:
link_video('videos/Chimpanzees1_reconstruction.mp4')